In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob, json, os, sys

sys.path.append('/mnt/llm/home/ibragim-bad/code/SWE-bench/swebench/metrics')
sys.path.append('/mnt/llm/home/ibragim-bad/code/SWE-bench/') # TODO: Replace with path to `SWE-bench/swebench/metrics` folder
from conversion import convert_log_to_ground_truth
from getters import get_logs_gold
from monitor import monitor_validation, monitor_logs_same_diff
sys.path = sys.path[:-1]

sys.path.append('/mnt/llm/home/ibragim-bad/code/SWE-bench/swebench/harness') # TODO: Replace with path to `SWE-bench/swebench/harness` folder
from utils import has_attribute_or_import_error
sys.path = sys.path[:-1]

/mnt/llm/home/ibragim-bad/code/conda/envs/swe-bench/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Declare repository; Fetch tasks, logs

In [3]:
repo = 'Farama-Foundation/Minigrid' # TODO: Replace with repository name
log_dir = '/mnt/llm/home/ibragim-bad/code/SWE-bench/data/log_dir' # TODO: Replace with path to folder of execution logs

Get map of version to setup commit

In [4]:
tasks = json.load(open('/mnt/llm/home/ibragim-bad/code/SWE-bench/data/testbed/versions/Minigrid-task-instances_versions.json')) # TODO: Replace with path to versioned candidate task instances
tasks = sorted(tasks, key=lambda x: x['created_at'], reverse=True)
version_to_setup_commit = {}
for t in tasks:
    if 'version' in t and t['version'] not in version_to_setup_commit:
        version_to_setup_commit[t['version']] = t['base_commit']
assert(
    sorted(list([x or "" for x in version_to_setup_commit.keys()])) ==
    sorted(list(set([t['version'] or "" for t in tasks if 'version' in t])))
)
tasks = {t['instance_id']: t for t in tasks}

#### Monitor Validation

In [5]:
failed_install, corrupt_test_patch, corrupt_patch, timeout, success = monitor_validation(log_dir)

Total Attempts: 19
Failed: 18
Usable: 1
Corrupt Test: 0
Corrupt Diff: 0
Test Script Timeout: 0
Success E2E: 1


In [16]:
logs_same, logs_diff = monitor_logs_same_diff(log_dir)
print(f"Logs same: {len(logs_same)}")
print(f"Logs diff: {len(logs_diff)}")

Logs same: 0
Logs diff: 1


#### Get [FP]2[FP] Tests

In [17]:
inst_id_to_gt = {}
for d in logs_diff:
    status_gt = convert_log_to_ground_truth(d[0])
    inst_id_to_gt[d[0]] = status_gt

#### Create Task Instances `.json` file

In [25]:
f'{t["instance_id"]}.log'

'Farama-Foundation__Minigrid-304.log'

In [24]:
t["repo"].split("/")[-1]

'Minigrid'

In [28]:
os.path.join(log_dir, f'{t["instance_id"]}.log')

'/mnt/llm/home/ibragim-bad/code/SWE-bench/data/log_dir/Farama-Foundation__Minigrid-304.log'

In [41]:
task['instance_id']

'Farama-Foundation__Minigrid-381'

In [39]:
t

{'repo': 'Farama-Foundation/Minigrid',
 'pull_number': 304,
 'instance_id': 'Farama-Foundation__Minigrid-304',
 'issue_numbers': ['303'],
 'base_commit': 'b7d447b34fc05565b05a28304e5b981a0a714b05',
 'patch': 'diff --git a/minigrid/core/mission.py b/minigrid/core/mission.py\n--- a/minigrid/core/mission.py\n+++ b/minigrid/core/mission.py\n@@ -172,14 +172,18 @@ def __eq__(self, other) -> bool:\n             # Check that place holder lists are the same\n             if self.ordered_placeholders is not None:\n                 # Check length\n-                if (len(self.order_placeholder) == len(other.order_placeholder)) and (\n+                if (\n+                    len(self.ordered_placeholders) == len(other.ordered_placeholders)\n+                ) and (\n                     all(\n                         set(i) == set(j)\n-                        for i, j in zip(self.order_placeholder, other.order_placeholder)\n+                        for i, j in zip(\n+                          

In [42]:
tasks_final = []
get_id_from_log = lambda x: x.split('/')[-1].split('.')[0]
for k, v in inst_id_to_gt.items():
    if len(v['FAIL_TO_PASS']) == 0:
        continue
    task = tasks[get_id_from_log(k)]
    task['FAIL_TO_PASS'] = v['FAIL_TO_PASS']
    task['PASS_TO_PASS'] = v['PASS_TO_PASS']
    task['environment_setup_commit'] = version_to_setup_commit[task['version']]

    # Do not consider tasks where the log before the patch has an attribute/import error
    log_path = os.path.join(log_dir, f'{task["instance_id"]}.log')
    log_before, log_after = get_logs_gold(log_path)
    if has_attribute_or_import_error(log_before):
        continue

    tasks_final.append(task)
print(f"Final number of task instances: ", len(tasks_final))

SAVE_PATH = "/mnt/llm/home/ibragim-bad/code/SWE-bench/data/minigrid_results.json" # TODO: Replace this with a path to a .json file to save the task instances
with open(SAVE_PATH, 'w') as f:
    json.dump(tasks_final, fp=f)

Final number of task instances:  1
